# 词袋模型


词袋模型（Bag-of-words model）是个在自然语言处理和信息检索(IR)下被简化的表达模型。这种模型的特点是字典中的字没有特定的顺序，句子的总体结构也被舍弃了，词袋模型中的one-hot、TF-IDF和n-gram文本表示方法。
词袋（Bag-of-words）是描述文档中单词出现的文本的一种表示形式。它涉及两件方面：（1）已知词汇的集合。（2）测试已知单词的存在。
因为文档中单词是以没有逻辑的顺序的放置，所以称为单词的“袋子”。该模型只关注文档中是否出现已知的单词，并不关注文档中的出现的单词。


# one-hot
对文本进行数值表示，那么先构造一个m×n的零矩阵，然后找到每个字在字典中出现的位置，把该位置的0替换为1，第二句话也这样处理。只管字出现了没有（出现了就填入1，不然就是0），而不管这个字在句子中出现了几次。one-hot编码后文本长度都一样了，方便进行矩阵运算。
one-hot存在以下问题：
（1）数据稀疏和维度灾难。数据稀疏也就是向量的大部分元素为0，如果词袋中的字词达数百万个，那么由每篇文档转换成的向量的维度是数百万维，由于每篇文档去重后字数较少，因此向量中大部分的元素是0。而且对数百万维的向量进行计算很困难。
（2）是没有考虑句中字的顺序性，假定字之间相互独立。这意味着意思不同的句子可能得到一样的向量。
（3）没有考虑字的相对重要性。这种表示只管字出现没有，而不管出现的频率，但显然一个字出现的次数越多，一般而言越重要（除了一些没有实际意义的停用词）。


# n-gram

上面词袋模型的两种表示方法假设字与字之间是相互独立的，没有考虑它们之间的顺序。于是引入n-gram（n元语法）的概念。n-gram是从一个句子中提取n个连续的字的集合，可以获取到字的前后信息，一般用2-gram或者3-gram。这种表示方法的好处是可以获取更丰富的特征，提取字的前后信息，考虑了字之间的顺序性。但是问题也是显而易见的，这种方法没有解决数据稀疏和词表维度过高的问题，而且随着n的增大，词表维度会变得更高。

总之，用词袋模型对文本的离散表示存在着数据稀疏、向量维度过高、字词之间的关系无法度量的问题，适用于浅层的机器学习模型，不适用于深度学习模型。


# 分布式向量
分布式词向量(distributed word representation)直接用普通的向量来表示词向量，而元素的值为任意实数，该向量的维数可以在事前确定，一般可以为50维或100维。这时的词向量类似如下（这里假设用5维来表示）：

中国 [1.25, 0.2, 0.3, 0.5, 0.6]

美国 [0.1, 0.3, 0.5, 0.1, 1.5]

日本 [2.2, 0.2, 0.4, 0.6, 1.0]

其中每个元素的具体数值则由训练来确定。这样一来就克服了在深度学习中可能的维度灾难，而且充分利用了空间，如果使用适当的训练方法训练出来的词向量可以直接根据两个词之间的距离来计算相似性。

# word2vec
谷歌开源的word2vec能生成词向量，通过词向量可以很好地度量词与词之间的相似性。word2vec采用的模型包含了连续词袋模型（CBOW）和Skip-Gram模型。通过它可以在大数据量上进行高效训练从而得到词向量。

一、神经网络语言模型
神经网络语言模型即用神经网络来训练语言模型，最经典的模型是Bengio等人提出的三层神经网络，对于语料C任意一个词w，取它的前n-1个词作为输入，这个跟n-gram的思路是一样的，而w则为它的输出，有了输入和输出就组成了训练样本了。神经网络语言模型解决了概率平滑、数据稀疏及维度灾难等问题，但它仍然无法彻底解决n的问题，即通过前n个词来预测接着的单词，这里认为神经网络语言模型无法取n大于5。另外一个是训练需要花费很多时间。

为了克服这些缺点，出现了CBOW模型和Skip-gram模型。它们都是通过对原始的神经网络语言模型进行改良。CBOW模型和Skip-gram模型的输入向量中不再使用one-hot形式，而是可以假设一个m维向量，初始值可以随机设置，然后通过训练不断优化最后得到具体的向量值。在训练CBOW模型和Skip-gram模型时会产生一个词向量的副产品，该词向量是一个分布式词向量。

二、CBOW模型
CBOW模型对原始的神经网络语言模型做了一些改造，比如把原来的隐含层去掉了，投影层直接与输出层相连，这是因为隐含层增加了计算量，而且去掉后基本不会影响效果。投影层做的操作就是累加输入层的所有向量，再回想一下传统神经网络语言模型是怎么处理输入层的？就是向量的连接操作，将输入层的所有向量首尾相连接。这种方式也就意味着丢弃词语的序列信息，这很好理解，原来方式是输入向量首尾先连，而现在是做累和操作。
CBOW模型训练其实就是根据某个词前后若干词来预测该词，这其实可以看成是多分类。最朴素的想法就是直接使用softmax来分别计算每个词的对应的归一化的概率。但对于动辄十几万词汇量的场景中使用softmax计算量太大，于是需要用一种二分类组合形式的hierarchical softmax，即输出层为一棵二叉树。

如何训练这棵二叉树。每个二叉树叶子结点表示一个词，非叶子结点则代表词，但它有一个权重参数θ，每个非叶子结点的权重参数都不同，另外再约定每个节点的左边的子节点编码为1，右边的子节点编码为0。那么从根节点开始（根节点不作编码）。

三、Skip-gram模型
Skip-Gram模型和CBOW的思路是反着来的，即输入是特定的一个词的词向量，而输出是特定词对应的上下文词向量。假设上下文大小取值为4， 特定的这个词"Learning"是输入，8个上下文词是输出。在这个Skip-Gram的例子里，输入是特定词， 输出是softmax概率排前8的8个词，对应的Skip-Gram神经网络模型输入层有1个神经元，输出层是词汇表大小个的神经元。隐藏层的神经元个数可以自己指定。通过DNN的反向传播算法，可以求出DNN模型的参数，同时得到所有的词对应的词向量。当有新的需求，要求出某个词对应的最可能的8个上下文词时，可以通过一次DNN前向传播算法得到概率大小排前8的softmax概率对应的神经元所对应的词即可。
